# Comparator Performance Analysis

This notebook helps you evaluate whether your comparator is a performance bottleneck and provides data-driven recommendations for optimization.

## How to Use

1. **Run the setup cells** to install dependencies and import modules
2. **Define your comparator** in the "Your Comparator" section
3. **Configure your workload** parameters
4. **Run all cells** to get analysis and recommendations

---

## 1. Setup

In [ ]:
# Core imports
import time
import random
import statistics
from typing import Callable, Any
from dataclasses import dataclass
from collections import defaultdict

# Visualization
try:
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    HAS_MATPLOTLIB = True
except ImportError:
    print("Installing matplotlib...")
    !pip install matplotlib -q
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    HAS_MATPLOTLIB = True

# For nicer output
try:
    import pandas as pd
    HAS_PANDAS = True
except ImportError:
    print("Installing pandas...")
    !pip install pandas -q
    import pandas as pd
    HAS_PANDAS = True

# concurrent_collections (comment out if not installed yet)
try:
    from concurrent_collections import SkipListMap, Comparator
    from concurrent_collections.instrumentation import ComparatorProfiler
    HAS_CC = True
except ImportError:
    HAS_CC = False
    print("⚠️  concurrent_collections not installed.")
    print("   This notebook will use simulation mode.")
    print("   Install with: pip install concurrent_collections")

print("✅ Setup complete")

In [ ]:
# Simulation mode fallback (when concurrent_collections not installed)
# This simulates the profiler behavior for notebook demonstration

if not HAS_CC:
    @dataclass
    class SimulatedReport:
        wall_time_sec: float
        comparator_time_sec: float
        comparator_time_pct: float
        comparison_count: int
        avg_comparison_ns: float
        p50_comparison_ns: float
        p95_comparison_ns: float
        p99_comparison_ns: float
        p999_comparison_ns: float
        max_comparison_ns: float
        
    class SimulatedProfiler:
        def __init__(self, cmp_func, n_comparisons=100000):
            self.cmp_func = cmp_func
            self.n_comparisons = n_comparisons
            self.latencies = []
            self.report = None
            
        def __enter__(self):
            return self
            
        def __exit__(self, *args):
            pass
            
        def run_benchmark(self, test_data):
            """Run comparison benchmark and collect latencies."""
            self.latencies = []
            
            start_total = time.perf_counter()
            cmp_time = 0
            
            for i in range(min(self.n_comparisons, len(test_data) - 1)):
                a, b = test_data[i], test_data[(i + 1) % len(test_data)]
                
                start = time.perf_counter_ns()
                _ = self.cmp_func(a, b)
                elapsed_ns = time.perf_counter_ns() - start
                
                self.latencies.append(elapsed_ns)
                cmp_time += elapsed_ns
            
            wall_time = time.perf_counter() - start_total
            cmp_time_sec = cmp_time / 1e9
            
            sorted_lat = sorted(self.latencies)
            n = len(sorted_lat)
            
            self.report = SimulatedReport(
                wall_time_sec=wall_time,
                comparator_time_sec=cmp_time_sec,
                comparator_time_pct=(cmp_time_sec / wall_time * 100) if wall_time > 0 else 0,
                comparison_count=len(self.latencies),
                avg_comparison_ns=statistics.mean(self.latencies) if self.latencies else 0,
                p50_comparison_ns=sorted_lat[int(n * 0.50)] if n > 0 else 0,
                p95_comparison_ns=sorted_lat[int(n * 0.95)] if n > 0 else 0,
                p99_comparison_ns=sorted_lat[int(n * 0.99)] if n > 0 else 0,
                p999_comparison_ns=sorted_lat[int(n * 0.999)] if n > 0 else 0,
                max_comparison_ns=max(self.latencies) if self.latencies else 0,
            )
            return self.report
    
    print("✅ Simulation mode enabled")

## 2. Your Comparator

Define the comparator you want to evaluate. Modify the `my_comparator` function below.

In [ ]:
# ============================================================
# ⬇️ MODIFY THIS: Define your custom comparator
# ============================================================

def my_comparator(a, b) -> int:
    """
    Your custom comparator function.
    
    Returns:
        negative if a < b
        zero if a == b  
        positive if a > b
    """
    # Example: Case-insensitive string comparison
    a_lower = a.lower()
    b_lower = b.lower()
    return (a_lower > b_lower) - (a_lower < b_lower)

# ============================================================
# Baseline comparator (natural ordering) for comparison
# ============================================================

def natural_comparator(a, b) -> int:
    """Standard Python comparison."""
    return (a > b) - (a < b)

print("✅ Comparators defined")
print(f"   my_comparator: {my_comparator.__doc__.strip().split(chr(10))[0] if my_comparator.__doc__ else 'Custom comparator'}")

## 3. Workload Configuration

Configure parameters that match your real-world usage.

In [ ]:
# ============================================================
# ⬇️ MODIFY THIS: Configure your workload
# ============================================================

# Data type you're comparing
DATA_TYPE = "string"  # Options: "string", "integer", "float", "custom"

# Number of items in your container (affects comparisons per operation)
CONTAINER_SIZE = 100_000

# Number of operations to benchmark
N_OPERATIONS = 50_000

# String length (if DATA_TYPE == "string")
STRING_LENGTH = 20

# Your expected operations per second in production
EXPECTED_OPS_PER_SEC = 10_000

# ============================================================
# Generate test data
# ============================================================

def generate_test_data(data_type: str, n: int) -> list:
    """Generate test data based on type."""
    if data_type == "string":
        chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        return ["".join(random.choices(chars, k=STRING_LENGTH)) for _ in range(n)]
    elif data_type == "integer":
        return [random.randint(-1_000_000, 1_000_000) for _ in range(n)]
    elif data_type == "float":
        return [random.uniform(-1000.0, 1000.0) for _ in range(n)]
    else:
        # Custom: override this for your data type
        return list(range(n))

test_data = generate_test_data(DATA_TYPE, max(CONTAINER_SIZE, N_OPERATIONS))
print(f"✅ Generated {len(test_data):,} test items of type '{DATA_TYPE}'")
print(f"   Sample: {test_data[0]!r}")

## 4. Run Benchmarks

Compare your comparator against the baseline.

In [ ]:
# Benchmark configuration
N_COMPARISONS = min(N_OPERATIONS * 20, 500_000)  # ~log2(N) comparisons per op

print(f"Running {N_COMPARISONS:,} comparisons per comparator...")
print()

results = {}

# Benchmark each comparator
comparators = {
    "Your Comparator": my_comparator,
    "Natural (baseline)": natural_comparator,
}

for name, cmp_func in comparators.items():
    print(f"Benchmarking: {name}...")
    
    if HAS_CC:
        # Use real profiler
        # Note: This would require actual container operations
        # For now, use simulation
        profiler = SimulatedProfiler(cmp_func, N_COMPARISONS)
    else:
        profiler = SimulatedProfiler(cmp_func, N_COMPARISONS)
    
    report = profiler.run_benchmark(test_data)
    results[name] = {
        "report": report,
        "latencies": profiler.latencies,
    }
    
    print(f"   ✓ {report.comparison_count:,} comparisons in {report.wall_time_sec:.3f}s")

print()
print("✅ Benchmarks complete")

## 5. Results Summary

In [ ]:
# Build comparison table
rows = []
baseline_avg = results["Natural (baseline)"]["report"].avg_comparison_ns

for name, data in results.items():
    r = data["report"]
    overhead = (r.avg_comparison_ns / baseline_avg - 1) * 100 if baseline_avg > 0 else 0
    
    rows.append({
        "Comparator": name,
        "Avg (ns)": f"{r.avg_comparison_ns:,.0f}",
        "P50 (ns)": f"{r.p50_comparison_ns:,.0f}",
        "P95 (ns)": f"{r.p95_comparison_ns:,.0f}",
        "P99 (ns)": f"{r.p99_comparison_ns:,.0f}",
        "Max (ns)": f"{r.max_comparison_ns:,.0f}",
        "Overhead": f"{overhead:+.0f}%" if name != "Natural (baseline)" else "baseline",
    })

df = pd.DataFrame(rows)
print("\n" + "="*80)
print("LATENCY COMPARISON")
print("="*80)
display(df)

In [ ]:
# Calculate production impact
your_report = results["Your Comparator"]["report"]
baseline_report = results["Natural (baseline)"]["report"]

# Estimate comparisons per operation (log2 of container size)
import math
comparisons_per_op = int(math.log2(CONTAINER_SIZE)) + 5  # +5 for overhead

# Time spent in comparisons per second at expected load
comparisons_per_sec = EXPECTED_OPS_PER_SEC * comparisons_per_op
your_cmp_time_per_sec = comparisons_per_sec * your_report.avg_comparison_ns / 1e9
baseline_cmp_time_per_sec = comparisons_per_sec * baseline_report.avg_comparison_ns / 1e9

print("\n" + "="*80)
print("PRODUCTION IMPACT ESTIMATE")
print("="*80)
print(f"\nWorkload assumptions:")
print(f"  Container size:     {CONTAINER_SIZE:,} items")
print(f"  Operations/sec:     {EXPECTED_OPS_PER_SEC:,}")
print(f"  Comparisons/op:     ~{comparisons_per_op} (log₂({CONTAINER_SIZE:,}))")
print(f"  Comparisons/sec:    {comparisons_per_sec:,}")
print()
print(f"Time in comparisons per second:")
print(f"  Your comparator:    {your_cmp_time_per_sec*1000:.2f} ms/sec ({your_cmp_time_per_sec*100:.1f}% of 1 CPU)")
print(f"  Natural ordering:   {baseline_cmp_time_per_sec*1000:.2f} ms/sec ({baseline_cmp_time_per_sec*100:.1f}% of 1 CPU)")
print(f"  Additional cost:    {(your_cmp_time_per_sec - baseline_cmp_time_per_sec)*1000:.2f} ms/sec")

## 6. Visualizations

In [ ]:
# Latency distribution histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#2ecc71', '#3498db']

for idx, (name, data) in enumerate(results.items()):
    ax = axes[idx]
    latencies = data["latencies"]
    
    # Cap at P99 for visualization
    p99 = sorted(latencies)[int(len(latencies) * 0.99)]
    display_latencies = [l for l in latencies if l <= p99 * 1.5]
    
    ax.hist(display_latencies, bins=50, color=colors[idx], alpha=0.7, edgecolor='black')
    ax.axvline(data["report"].p50_comparison_ns, color='red', linestyle='--', 
               label=f'P50: {data["report"].p50_comparison_ns:,.0f} ns')
    ax.axvline(data["report"].p95_comparison_ns, color='orange', linestyle='--',
               label=f'P95: {data["report"].p95_comparison_ns:,.0f} ns')
    ax.axvline(data["report"].p99_comparison_ns, color='purple', linestyle='--',
               label=f'P99: {data["report"].p99_comparison_ns:,.0f} ns')
    
    ax.set_xlabel('Latency (ns)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{name}\nAvg: {data["report"].avg_comparison_ns:,.0f} ns')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Comparison Latency Distribution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Percentile comparison bar chart
fig, ax = plt.subplots(figsize=(10, 6))

percentiles = ['P50', 'P95', 'P99', 'P99.9']
x = range(len(percentiles))
width = 0.35

your_values = [
    your_report.p50_comparison_ns,
    your_report.p95_comparison_ns,
    your_report.p99_comparison_ns,
    your_report.p999_comparison_ns,
]

baseline_values = [
    baseline_report.p50_comparison_ns,
    baseline_report.p95_comparison_ns,
    baseline_report.p99_comparison_ns,
    baseline_report.p999_comparison_ns,
]

bars1 = ax.bar([i - width/2 for i in x], your_values, width, label='Your Comparator', color='#3498db')
bars2 = ax.bar([i + width/2 for i in x], baseline_values, width, label='Natural (baseline)', color='#2ecc71')

ax.set_xlabel('Percentile')
ax.set_ylabel('Latency (ns)')
ax.set_title('Latency Percentile Comparison')
ax.set_xticks(x)
ax.set_xticklabels(percentiles)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:,.0f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:,.0f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Tail latency analysis
fig, ax = plt.subplots(figsize=(10, 5))

# Check for tail latency issues
your_tail_ratio = your_report.p99_comparison_ns / your_report.p50_comparison_ns
baseline_tail_ratio = baseline_report.p99_comparison_ns / baseline_report.p50_comparison_ns

ratios = {
    'Your Comparator': your_tail_ratio,
    'Natural (baseline)': baseline_tail_ratio,
}

colors_tail = ['#e74c3c' if r > 10 else '#f39c12' if r > 5 else '#2ecc71' for r in ratios.values()]

bars = ax.bar(ratios.keys(), ratios.values(), color=colors_tail, edgecolor='black')
ax.axhline(y=5, color='orange', linestyle='--', label='Warning threshold (5x)')
ax.axhline(y=10, color='red', linestyle='--', label='Critical threshold (10x)')

ax.set_ylabel('P99 / P50 Ratio')
ax.set_title('Tail Latency Ratio (P99 / P50)\nHigher = more latency variability')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

for bar, ratio in zip(bars, ratios.values()):
    ax.annotate(f'{ratio:.1f}x',
                xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Recommendations

In [ ]:
# Generate recommendations based on analysis

def generate_recommendations(your_report, baseline_report, comparisons_per_sec):
    """Generate actionable recommendations based on benchmark results."""
    recommendations = []
    
    # Calculate key metrics
    overhead_pct = (your_report.avg_comparison_ns / baseline_report.avg_comparison_ns - 1) * 100
    overhead_ns = your_report.avg_comparison_ns - baseline_report.avg_comparison_ns
    tail_ratio = your_report.p99_comparison_ns / your_report.p50_comparison_ns
    time_per_sec = comparisons_per_sec * your_report.avg_comparison_ns / 1e9
    
    # Severity levels
    severity = "✅ LOW"
    if time_per_sec > 0.3:  # >30% of 1 CPU
        severity = "🔴 CRITICAL"
    elif time_per_sec > 0.1:  # >10% of 1 CPU
        severity = "🟠 HIGH"
    elif time_per_sec > 0.05:  # >5% of 1 CPU
        severity = "🟡 MODERATE"
    
    print("="*80)
    print(f"RECOMMENDATIONS                                      Severity: {severity}")
    print("="*80)
    
    # Overhead analysis
    print(f"\n📊 OVERHEAD ANALYSIS")
    print(f"   Your comparator is {overhead_pct:+.0f}% vs natural ordering ({overhead_ns:+.0f} ns/comparison)")
    
    if overhead_pct < 50:
        print(f"   ✅ Overhead is acceptable for most use cases")
    elif overhead_pct < 200:
        print(f"   ⚠️  Moderate overhead - consider optimization if this is a hot path")
    else:
        print(f"   🔴 High overhead - strongly consider optimization")
    
    # Tail latency analysis
    print(f"\n📈 TAIL LATENCY ANALYSIS")
    print(f"   P99/P50 ratio: {tail_ratio:.1f}x")
    
    if tail_ratio < 3:
        print(f"   ✅ Consistent latency - no tail latency issues")
    elif tail_ratio < 10:
        print(f"   ⚠️  Some latency variability - may be GC or contention")
    else:
        print(f"   🔴 High tail latency - investigate GC, locks, or I/O in comparator")
    
    # Production impact
    print(f"\n🏭 PRODUCTION IMPACT")
    print(f"   At {EXPECTED_OPS_PER_SEC:,} ops/sec: {time_per_sec*100:.1f}% of 1 CPU in comparisons")
    
    if time_per_sec < 0.01:
        print(f"   ✅ Negligible impact - no optimization needed")
    elif time_per_sec < 0.05:
        print(f"   ✅ Low impact - current implementation is fine")
    elif time_per_sec < 0.1:
        print(f"   🟡 Moderate impact - consider key function if possible")
    elif time_per_sec < 0.3:
        print(f"   🟠 Significant impact - recommend native comparator")
    else:
        print(f"   🔴 Critical impact - native comparator strongly recommended")
    
    # Specific recommendations
    print(f"\n💡 SPECIFIC RECOMMENDATIONS")
    
    if time_per_sec < 0.05:
        print(f"   1. Keep your current Python comparator")
        print(f"   2. No optimization needed at this scale")
    elif overhead_pct > 100:
        print(f"   1. Consider using key= instead of cmp= if possible")
        print(f"      Example: SkipListMap(key=str.lower) instead of cmp=...")
        print(f"   2. Key functions extract once, compare many times")
    
    if time_per_sec > 0.1:
        print(f"   3. For maximum performance, implement a native comparator:")
        print(f"      - Cython: Best for Python-focused projects")
        print(f"      - Rust with C ABI: Best for new code")
        print(f"      - C: Best for existing C codebases")
    
    if tail_ratio > 5:
        print(f"   4. Investigate tail latency:")
        print(f"      - Check for GC pressure in comparator")
        print(f"      - Avoid creating temporary objects")
        print(f"      - Consider string interning for repeated comparisons")
    
    # Estimated improvement
    print(f"\n📉 ESTIMATED IMPROVEMENT WITH NATIVE COMPARATOR")
    native_estimate_ns = baseline_report.avg_comparison_ns * 1.2  # ~20% overhead for native
    native_time_per_sec = comparisons_per_sec * native_estimate_ns / 1e9
    improvement = (time_per_sec - native_time_per_sec) / time_per_sec * 100 if time_per_sec > 0 else 0
    
    print(f"   Current:  {time_per_sec*100:.1f}% CPU")
    print(f"   Estimated: {native_time_per_sec*100:.1f}% CPU (native)")
    print(f"   Potential improvement: ~{improvement:.0f}%")
    
    if improvement > 50:
        print(f"   ✅ Native comparator would be worthwhile")
    else:
        print(f"   ⚠️  Limited benefit from native comparator at this scale")

generate_recommendations(your_report, baseline_report, comparisons_per_sec)

## 8. Export Results

In [ ]:
# Export results for sharing
import json
from datetime import datetime

export_data = {
    "timestamp": datetime.now().isoformat(),
    "configuration": {
        "data_type": DATA_TYPE,
        "container_size": CONTAINER_SIZE,
        "n_operations": N_OPERATIONS,
        "expected_ops_per_sec": EXPECTED_OPS_PER_SEC,
    },
    "results": {}
}

for name, data in results.items():
    r = data["report"]
    export_data["results"][name] = {
        "avg_ns": r.avg_comparison_ns,
        "p50_ns": r.p50_comparison_ns,
        "p95_ns": r.p95_comparison_ns,
        "p99_ns": r.p99_comparison_ns,
        "p999_ns": r.p999_comparison_ns,
        "max_ns": r.max_comparison_ns,
        "comparison_count": r.comparison_count,
    }

# Save to file
output_file = "comparator_benchmark_results.json"
with open(output_file, "w") as f:
    json.dump(export_data, f, indent=2)

print(f"✅ Results exported to {output_file}")
print()
print("JSON preview:")
print(json.dumps(export_data, indent=2)[:500] + "...")

## 9. Next Steps

Based on the recommendations above:

### If optimization is NOT needed:
- Keep your current Python comparator
- Re-run this analysis if your workload changes significantly

### If key function is recommended:
```python
# Instead of:
m = SkipListMap(cmp=lambda a, b: (a.lower() > b.lower()) - (a.lower() < b.lower()))

# Use:
m = SkipListMap(key=str.lower)
```

### If native comparator is recommended:

See the [Comparator Design Documentation](../doc/design/tier-1/comparator/design.md) for:
- Cython implementation examples
- Rust with C ABI examples
- Pure C examples

### Re-run after optimization:
1. Implement your native comparator
2. Update the `my_comparator` cell to use it
3. Re-run all cells to verify improvement

In [ ]:
print("="*80)
print("Analysis complete!")
print("="*80)
print()
print("Summary:")
print(f"  Your comparator:    {your_report.avg_comparison_ns:,.0f} ns avg")
print(f"  Natural ordering:   {baseline_report.avg_comparison_ns:,.0f} ns avg")
print(f"  Overhead:           {(your_report.avg_comparison_ns / baseline_report.avg_comparison_ns - 1) * 100:+.0f}%")
print(f"  Production impact:  {comparisons_per_sec * your_report.avg_comparison_ns / 1e9 * 100:.1f}% CPU")